## text similarity with tfidf

In [17]:
# hyperparameters
# MS/LINUX glossary 的權重加分
keyword_weight_increase = 1
# 無母音的單字權重加分
keyword_weight_increase_NoVowel = 1

In [18]:
import numpy as np

In [19]:
with open("clean_corpus.txt", 'r', encoding='utf8') as f:
    clean_corpus=f.readlines()
    for i in range(len(clean_corpus)):
        clean_corpus[i]=clean_corpus[i].replace('\n', '')

with open("original_corpus.txt", 'r', encoding='utf8') as f:
    original_corpus=f.readlines()
    for i in range(len(original_corpus)):
        original_corpus[i]=original_corpus[i].replace('\n', ' ')        

In [20]:
import pickle
with open("keyword_list_NoVowel.pickle", 'rb') as f:
    keyword_list_NoVowel = pickle.load(f)

In [21]:
import pickle
with open("keyword_list.pickle", 'rb') as f:
    keyword_list_TERM = pickle.load(f)
#keyword_list_TERM.keys()

## compare corpus with n-gram keywords, n=2~4

In [22]:
def check_special_term(words, start_index, n_gram):
    merge_word=[]
    for i in range(n_gram):
        merge_word.append(words[start_index+i])
    if ' '.join(merge_word) in keyword_list_TERM:
        #print('converted_word=', keyword_list_TERM[' '.join(merge_word)]['converted_word'])
        return keyword_list_TERM[' '.join(merge_word)]['converted_word']
    else:
        return ''

def compare_corpus_with_keyword(clean_corpus):
    new_clean_corpus=[]
    merge_word_list=[] # n-gram keywords
    hit_row_index=[] # keep for predict test
    for no, line in enumerate(clean_corpus):
        words = line.split(' ')
        #words = word_tokenize(line) 
        words_len = len(words)
        # n-gram
        new_line=''
        for i in range(4, 1, -1):
            for j in range(words_len-i+1):
                merge_word = check_special_term(words, j, i)
                if merge_word == '':
                    new_line+=' '+words[j]
                    if j==words_len-i:
                        for k in range(j+1, words_len):
                            new_line+=' '+words[k]
                    continue
                else:
                    merge_word_list.append(merge_word)
                    hit_row_index.append(no)
                    new_line+=' '+merge_word
                    j+=i
                    continue

            # line is too short, keep it as original
            if words_len-i<0:
                new_line=line    
            new_line = new_line.strip()
            words = new_line.split(' ')
            words_len = len(words)
            if i>2:
                new_line=''
        new_clean_corpus.append(new_line)
        #if new_line != line:
        #    print('org_line='+line+'\n')
        #    print('new_line='+new_line+'\n\n')
    return new_clean_corpus, merge_word_list, hit_row_index

new_clean_corpus, merge_word_list, hit_row_index = compare_corpus_with_keyword(clean_corpus)
hit_row_index=list(set(hit_row_index))        

In [23]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(new_clean_corpus)
print (tfidf.shape)

words = vectorizer.get_feature_names()


(1719, 8709)


In [24]:
# test if keyword exists
'visual_c' in vectorizer.vocabulary_.keys()

False

merge_word_list=list(set(merge_word_list))
#merge_word_list

In [25]:
keyword_list_TERM_converted=[]
for item in keyword_list_TERM.keys():
    keyword_list_TERM_converted.append(keyword_list_TERM[item]['converted_word'])

In [26]:
# change weight of keywords for training data manually
def adjust_weight_based_on_keyword(tfidf_1):
    # adjust weight based on terminology
    for key_term in keyword_list_TERM_converted:
        if not key_term in vectorizer.vocabulary_.keys():
            continue
        key_term_code = vectorizer.vocabulary_[key_term]
        #if key_term == 'exchange_server':
        #    print(key_term_code)
        for row in range(tfidf_1.toarray().shape[0]):
            if tfidf_1[row, key_term_code] > 1e-5:
                tfidf_1[row,key_term_code]+=keyword_weight_increase
                
    # adjust weight based on products/services acronym
    for key_term in keyword_list_NoVowel.keys():
        if not key_term in vectorizer.vocabulary_.keys():
            continue
        key_term_code = vectorizer.vocabulary_[key_term]
        #if key_term == 'exchange_server':
        #    print(key_term_code)
        for row in range(tfidf_1.toarray().shape[0]):
            if tfidf_1[row, key_term_code] > 1e-5:
                tfidf_1[row,key_term_code]+=keyword_weight_increase_NoVowel
                
adjust_weight_based_on_keyword(tfidf)        

### get adjusted weights of 'exchange_server'
```
# adjusted weights of 'exchange_server'
for row in range(tfidf.toarray().shape[0]):
    if tfidf[row, 3745] > 1e-5:
        print(tfidf[row,3745])
```

In [27]:
# 任選一句預測
document2 = [new_clean_corpus[hit_row_index[0]]]

new_data = vectorizer.transform(document2)
adjust_weight_based_on_keyword(new_data)  

#print(new_data.toarray())
#print ('-'*20)

# print top 10 keywords
top10_index_list=np.argsort(new_data.toarray()[0])[::-1][:11]
for j in range(len(top10_index_list)):
    if new_data[0,top10_index_list[j]]> 1e-5:
        print (words[top10_index_list[j]], new_data[0,top10_index_list[j]])

works 1.2646511321858687
exchange_server 1.2284476755999365
wfb 1.1302115289712866
security 1.1139505071005387
agent 1.111182488361652
server 1.0993284265809409
smex 0.6230860887022853
2010 0.29013768958395175
master 0.2532055301506258
hour 0.24618505246920577
exchange 0.23824448377043922


In [28]:
new_data.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [31]:
original_corpus[hit_row_index[0]]

'WFBS 10 exchange Messaging Security Agent not working SMEX   Exchange server 2010 SP3 when restart smex master.exe  it works again for 24 hours '

In [33]:
# 顯示與其他句的相似度
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similar_prob=cosine_similarity(new_data, tfidf.toarray())
print (similar_prob[0])

similar_corpus_index = np.argsort(similar_prob[0])[::-1][:11]
print(similar_corpus_index)
print("new data 原文:[{}]".format(original_corpus[hit_row_index[0]]))
print("new data 處理後:", document2)
for i in range(len(similar_corpus_index)):
    #print('{}:{}\n'.format(i, new_clean_corpus[similar_corpus_index[i]]))
    print('{}:{}\n'.format(i, original_corpus[similar_corpus_index[i]]))

[0.0033793  0.         0.         ... 0.         0.28296256 0.        ]
[ 384 1607  150  343  448  290  678   57  391 1324  332]
new data 原文:[WFBS 10 exchange Messaging Security Agent not working SMEX   Exchange server 2010 SP3 when restart smex master.exe  it works again for 24 hours ]
new data 處理後: ['wfb exchange message security agent working smex exchange_server server 2010 sp3 restart smex master works hour']
0:WFBS 10 exchange Messaging Security Agent not working SMEX   Exchange server 2010 SP3 when restart smex master.exe  it works again for 24 hours 

1:Multiple users get this message when they receive email from @listserv.cuis.edu.  Administrators of the listserv report no issues. What would be triggering the detection? Is there a way to whitelist in WFBS Standard?  (We do not use exchange server.)  Message received: "Subject: Trend Micro OfficeScan detected and took action on a malicious email  Trend Micro Worry-Free Business Security Agent has detected an email that may cont